In [26]:
%load_ext autoreload
%autoreload 2

#standard library imports
import re
import nltk
import numpy as np
import pandas as pd
import os

#modeling functions & utilities
from pronounResolution import pronResolution_base, pronEval
from relationExtract import simpleRE, REEval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
files = [x for x in os.listdir('prep_scripts') if '_gapi' in x]
df = pd.read_csv('prep_scripts/' + files[0])[['speaker', 'dialogue', 'sentences', 'sentiment', 'entities', 'tokens']]
df['tokens'] = df['tokens'].apply(lambda x: eval(x))
df['sentiment'] = df['sentiment'].apply(lambda x: eval(x))
df.head(10)

,speaker,dialogue,sentences,sentiment,entities,tokens
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'score': 0.3, 'magnitude': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'content': '1989', 'index': 0, 'lemma': '198..."
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'score': 0.1, 'magnitude': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'content': 'Stark', 'index': 0, 'lemma': 'St..."
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'score': -0.6, 'magnitude': 0.6}",[],"[{'content': 'He', 'index': 3, 'lemma': 'He', ..."
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'score': -0.1, 'magnitude': 1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'content': 'Hello', 'index': 2, 'lemma': 'He..."
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{'score': 0.4, 'magnitude': 0.4}","[{'salience': 0.34197578, 'meta': {}, 'type': ...","[{'content': 'I', 'index': 1, 'lemma': 'I', 'b..."
5,Peggy Carter,Tell me that isn't what I think it is.,"[{'content': u""Tell me that isn't what I think...","{'score': -0.6, 'magnitude': 0.6}",[],"[{'content': 'Tell', 'index': 0, 'lemma': 'Tel..."
6,Dr. Hank Pym,"It depends, if you think it's a poor attempt t...","[{'content': u""It depends, if you think it's a...","{'score': -0.5, 'magnitude': 1}","[{'salience': 0.88545346, 'meta': {}, 'type': ...","[{'content': 'It', 'index': 1, 'lemma': 'It', ..."
7,Mitchell Carson,You were instructed to go to Russia. May I rem...,[{'content': u'You were instructed to go to Ru...,"{'score': 0, 'magnitude': 0.7}","[{'salience': 0.87446207, 'meta': {}, 'type': ...","[{'content': 'You', 'index': 2, 'lemma': 'You'..."
8,Dr. Hank Pym,I'm a scientist.,"[{'content': u""I'm a scientist."", 'begin': 0, ...","{'score': 0.3, 'magnitude': 0.3}","[{'salience': 1, 'meta': {}, 'type': u'PERSON'...","[{'content': 'I', 'index': 1, 'lemma': 'I', 'b..."
9,Howard Stark,Then act like one. The Pym Particle is the mos...,"[{'content': u'Then act like one.', 'begin': 0...","{'score': 0.1, 'magnitude': 1.2}","[{'salience': 0.6887065, 'meta': {}, 'type': u...","[{'content': 'Then', 'index': 1, 'lemma': 'The..."


In [124]:
cList = list(df.speaker.unique())
df['total_sent'] = df['sentiment'].apply(lambda x: x['score'] * x['magnitude'])
cDict = dict(df.groupby('speaker').total_sent.sum())
df.shift().head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'score': 0.3, 'magnitude': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'content': '1989', 'index': 0, 'lemma': '198...",0.09
2,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'score': 0.1, 'magnitude': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'content': 'Stark', 'index': 0, 'lemma': 'St...",0.01
3,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'score': -0.6, 'magnitude': 0.6}",[],"[{'content': 'He', 'index': 3, 'lemma': 'He', ...",-0.36
4,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'score': -0.1, 'magnitude': 1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'content': 'Hello', 'index': 2, 'lemma': 'He...",-0.10


In [125]:
df['tokens'], df['hasChar'] = zip(*df.apply(lambda x:pronResolution_base(cList, x), axis=1).values)
df.head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent,hasChar
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'score': 0.3, 'magnitude': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'content': '1989', 'index': 0, 'lemma': '198...",0.09,False
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'score': 0.1, 'magnitude': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'content': 'Stark', 'index': 0, 'lemma': 'St...",0.01,False
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'score': -0.6, 'magnitude': 0.6}",[],"[{'content': 'He', 'index': 3, 'lemma': 'He', ...",-0.36,True
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'score': -0.1, 'magnitude': 1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'content': 'Hello', 'index': 2, 'lemma': 'He...",-0.10,True
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{'score': 0.4, 'magnitude': 0.4}","[{'salience': 0.34197578, 'meta': {}, 'type': ...","[{'content': 'I', 'index': 1, 'lemma': 'I', 'c...",0.16,True


In [14]:
pronEval([df, df], num=2)

*******for the following dialogue********
Hope van Dyne: I don't know why I came to you in the first place. [Hope leaves]
Dr. Hank Pym: We can't do this without her.
=>narrator: as Hope is about to leave in her car Scott gets in<=
Hope van Dyne: Oh, God.
Scott Lang: You gotta lock your doors. I mean, really. There's some weird folks in this neighborhood.
********************
1 pronouns resolved
1. her => ['Pym Tech Employee']
how many are correctly identified?0
*******for the following dialogue********
narrator: Scott floats along on the water with the ants through the pipes
Kurt: Coming up on extraction pipe.
=>Scott Lang: I see it! [to the ants] Alright, come on. I gotta get up there. That's it, that's it, guys, yeah! [the ants help him climb up to the extraction pipe] That's it, yeah. Yes! You got it! You got it, come on! [Scott makes it through one of the handwash sinks in the bathroom] Alright, let's fly Antony. [Scott and the ants fly off]<=
Kurt: The Ant-Man is in the building.


In [126]:
df['relations'], df['hasRelation'] = zip(*df.apply(lambda x:simpleRE(x), axis=1))
df.head()

,speaker,dialogue,sentences,sentiment,entities,tokens,total_sent,hasChar,relations,hasRelation
0,narrator,1989 – Hank Pym enters a SHIELD facility,[{'content': u'1989 \u2013 Hank Pym enters a S...,"{'score': 0.3, 'magnitude': 0.3}","[{'salience': 0.5188706, 'meta': {}, 'type': u...","[{'content': '1989', 'index': 0, 'lemma': '198...",0.09,False,"[{'line': 0, 'class': 0, 'ent1': 'Pym', 'ent2'...",True
1,Dr. Hank Pym,Stark.,"[{'content': u'Stark.', 'begin': 0, 'score': 0...","{'score': 0.1, 'magnitude': 0.1}","[{'salience': 1, 'meta': {}, 'type': u'WORK_OF...","[{'content': 'Stark', 'index': 0, 'lemma': 'St...",0.01,False,None,False
2,Mitchell Carson,He doesn't seem happy.,"[{'content': u""He doesn't seem happy."", 'begin...","{'score': -0.6, 'magnitude': 0.6}",[],"[{'content': 'He', 'index': 3, 'lemma': 'He', ...",-0.36,True,None,False
3,Howard Stark,"Hello, Hank. You're supposed to be in Moscow.","[{'content': u'Hello, Hank.', 'begin': 0, 'sco...","{'score': -0.1, 'magnitude': 1}","[{'salience': 0.80661523, 'meta': {}, 'type': ...","[{'content': 'Hello', 'index': 2, 'lemma': 'He...",-0.10,True,None,False
4,Dr. Hank Pym,I took a detour.[he places a vial containing a...,[{'content': u'I took a detour.[he places a vi...,"{'score': 0.4, 'magnitude': 0.4}","[{'salience': 0.34197578, 'meta': {}, 'type': ...","[{'content': 'I', 'index': 1, 'lemma': 'I', 'c...",0.16,True,"[{'line': 4, 'class': 0, 'ent1': ['Luis'], 'en...",True


In [27]:
REEval([df], 2)


******** line [313 952] ********
311. narrator:
he takes out the suit and looks at the helmet

312. Scott Lang:
So weird.

=> 313. narrator:
=> he tries the suit on and steps into the bathtub to get a better look at himself in the mirror

314. Luis:
Scott, what's up man?

315. narrator:
Scott closes the shower curtain then looks at the buttons on the gloves

******** test model 1: line 313 ********
1 relations identified
entities: ['Hope van Dyne'] => suit
relation: tries
category: 0

how many are correctly identified? 0

******** line [313 952] ********
950. Paxton:
She's fine. [Maggie enters the room]

951. Cassie Lang:
Mommy. [Maggie grabs hold of Cassie with relief]

=> 952. Paxton:
=> She's fine, she's fine. [Cassie notices Scott in his shrunken size wave goodbye to her and leaves]

953. narrator:
the next day Scott is over at Pym's house

954. Dr. Hank Pym:
Scott, please. You don't remember anything?

******** test model 1: line 952 ********
1 relations identified
entities: Cass

In [155]:
def checkQuery(relationList, ent1, ent2, relationClass):
    for relation in relationList:
        if ent1 in relation['ent1'] and ent2 in relation['ent2'] and relationClass == relation['class']:
            return True
    return False

def printAnswer(row):
    print('Movie: {}, Line {}'.format(row.movie, row.lineNum))
    print(row.dialogue)
    print()

#Simple Query System

print('Select the movies of your interest:')
print('***Enter all to use all movies')
print('***Enter n, m, x, y (numbers separated by commas) for specific selections')
print('***Enter random, n for n random selections\n')

files = [x for x in os.listdir('prep_scripts') if '_gapi' in x]
for i, fileName in enumerate(files):
    print('{}. {}'.format(i+1, re.split(r'_tw_|_imsdb_', fileName)[0]))


x = input()


#random selection
try:
    if 'random' in x:
        queryFiles = np.random.choice(files, int(x.split(',')[-1]), replace=False)
    elif x != 'all':
        queryFiles = np.array(files)[[int(select) - 1 for select in x.split(',')]]
    #use all files
    else:
        queryFiles = files    
        
except:
    print('\nunexpected input, will use all movie files\n')
    queryFiles = files    

#print(queryFiles)
df_data = None

for i, fileName in enumerate(queryFiles):    
    df = pd.read_csv('prep_scripts/'+fileName)[['speaker', 'dialogue', 'sentences', 'sentiment', 'entities', 'tokens']]
    df['tokens'] = df['tokens'].apply(lambda x: eval(x))
    df['sentiment'] = df['sentiment'].apply(lambda x: eval(x))
    df['total_sent'] = df['sentiment'].apply(lambda x: x['score'] * x['magnitude'])
    df['movie'] = re.split(r'_tw_|_imsdb_', fileName)[0]
    df['lineNum'] = df.index + 1
    
    cList = list(df.speaker.unique())
    cDict = dict(df.groupby('speaker').total_sent.sum())
    
    #resolve entities
    df['tokens'], df['hasChar'] = zip(*df.apply(lambda x:pronResolution_base(cList, x), axis=1).values)
    
    #extract relations
    df['relations'], df['hasRelation'] = zip(*df.apply(lambda x:simpleRE(x), axis=1))
    
    if i == 0:
        df_data = df[df.hasRelation == True]
        
    else:
        df_data = pd.concat((df_data, df[df.hasRelation == True]))

    
print('Type end to finish at any time')

#relationList = df_data[df_data.hasRelation == True]['relations'].values

while True:
    print('What relation are you looking for?')
    ent1 = input('Entity 1:')
    if ent1 == 'end':
        break
    ent2 = input('Entity 2:')
    if ent2 == 'end':
        break
    relationClass = int(input('Relation category: '))
    
    df_data[df_data.relations.apply(lambda x: checkQuery(x, ent1, ent2, relationClass))].apply(lambda x: printAnswer(x), axis=1)
    

Select the movies of your interest:
***Enter all to use all movies
***Enter n, m, x, y (numbers separated by commas) for specific selections
***Enter random, n for n random selections

1. ant-man
2. avengers_age_of_ultron
3. captain_america_civil_war
4. captain_america_the_first_avenger
5. captain_america_the_winter_soldier
6. fantastic_four
7. iron_man_3
8. lego_marvel_super_heroes
9. spider-man
10. the_amazing_spider-man_2
11. the_amazing_spider-man
12. the_avengers
13. the_wolverine
14. thor_the_dark_world
15. thor
16. x-men_apocalypse
17. x-men_days_of_future_past
18. x-men
19. x-men_the_last_stand
15
Type end to finish at any time
What relation are you looking for?
Entity 1:Frigga
Entity 2:Frigga
Relation category: 0
What relation are you looking for?
Entity 1:Thor
Entity 2:Loki
Relation category: 0
Movie: thor, Line 413
Good. In that case, I'll buy you another round. And you leave town tonight. [Thor nods his head]

Movie: thor, Line 502
[referring to Loki saving Odin from being 

In [143]:
df_data.tail().apply(lambda x: printAnswer(x), axis=1)

Movie: x-men_the_last_stand, Line 334
Help me? What's wrong with me?
Movie: x-men_the_last_stand, Line 335
Absolutely nothing.
Movie: x-men_the_last_stand, Line 336
Eric, stop.
Movie: x-men_the_last_stand, Line 337
No, Charles, not this time. You've always held her back.
Movie: x-men_the_last_stand, Line 338
For your own good, Jean.


333    None
334    None
335    None
336    None
337    None
dtype: object

In [154]:
df_data.relations.values[0]

[{'class': 0,
  'ent1': ['Admission Nurse'],
  'ent2': ['Volstagg'],
  'line': 7,
  'relation': 'telling'},
 {'class': 0,
  'ent1': ['Frigga'],
  'ent2': ['Frigga'],
  'line': 7,
  'relation': 'ask'}]